<a href="https://colab.research.google.com/github/ryzigor/portfolio/blob/main/%D0%A0%D1%8B%D0%B6%D0%BE%D0%B2%D0%98%D0%B3%D0%BE%D1%80%D1%8C%D0%9C%D0%B8%D1%85%D0%B0%D0%B9%D0%BB%D0%BE%D0%B2%D0%B8%D1%87__IgorRyzhov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#импорт библиотек и создание/подключение к базе
import pandas as pd
import numpy as np
import sqlite3
con = sqlite3.connect('/content/drive/MyDrive/data/db')

#Задача 1

In [ ]:
id = '1LdLXZmtGp9AijjtTkICERkRZ2iyBlaC_SBOZ1jB8cgw'
df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv')
#df = pd.read_csv('/content/drive/MyDrive/data/task1 sql context yandex.csv', sep=';')
df['date'] = pd.to_datetime(df['date'], dayfirst=True).dt.date
df.to_sql('total_sales', con, index=False, if_exists='replace')

7

In [ ]:
#функция для обертки запроса
def select(sql):
  return pd.read_sql(sql, con)

In [ ]:
sql = '''select 
  *, 
  min(date) over (partition by buyer_id) as first_deal,
  lag(date, 1, null) over (partition by buyer_id order by date) as last_deal,
  sum(qty) over (partition by buyer_id) as total_buyer_qty,
  row_number() over (partition by buyer_id, seller_id order by date) as deal_number
  from total_sales'''
select(sql)

,sales_id,date,buyer_id,seller_id,qty,qty_shipped,first_deal,last_deal,total_buyer_qty,deal_number
0,34489,2022-06-18,1,f,30,(null),2022-06-18,None,40,1
1,40494,2022-12-04,1,m,10,10,2022-06-18,2022-06-18,40,1
2,83056,2022-06-07,2,m,70,(null),2022-06-07,None,80,1
3,36044,2022-07-07,2,g,10,10,2022-06-07,2022-06-07,80,1
4,85754,2022-05-04,3,a,100,100,2022-05-04,None,100,1
5,96498,2022-07-19,4,e,70,70,2022-07-19,None,110,1
6,25651,2022-11-06,4,f,40,(null),2022-07-19,2022-07-19,110,1


#Задача 2

Импорт тестовых данных

In [ ]:
id = '1njU4CNYsiZxsA6fF1-hTUxNHOACVuSCxOhhkEInLRCs'
sales = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv')
sales['dt'] = pd.to_datetime(sales['dt'], dayfirst=True).dt.date
sales.to_sql('sales', con, index=False, if_exists='replace')
id = '126CMg-Ul7ftAbwhZGuD90VVvQRyvyPaPpE_34WMOChw'
stores = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv')
stores.to_sql('stores', con, index=False, if_exists='replace')
id ='1alMHHYNq-RoyVry3H9-36k7Ga7jVpKRNd4M9lClF-KU'
customers= pd.read_csv(f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv')
customers.to_sql('customers', con, index=False, if_exists='replace')


10

Запрос 1a

In [ ]:
sql = '''
SELECT c.customer_nm, s.store_nm
FROM
  (SELECT 
    customer_id, 
    store_id,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY COUNT (DISTINCT sale_id) DESC) AS shop_rate
  FROM sales
  GROUP BY 1,2) AS m
LEFT JOIN stores AS s ON m.store_id = s.store_id
LEFT JOIN customers c ON m.customer_id = c.customer_id 
WHERE shop_rate = 1'''
select(sql)

,customer_nm,store_nm
0,Матвей Кондратьев,Shingo
1,Дарина Жукова,Shingo
2,Иван Леонтьев,Practix
3,Аделина Макарова,Stanter
4,Марина Александрова,Practix
5,Юлия Белякова,Practix
6,Арина Афанасьева,Stanter
7,Светлана Литвинова,Shingo
8,Виктория Савельева,Stanter
9,Юрий Герасимов,Shingo


Запрос 1b

In [ ]:
#особенности работы с датой для SQLIte - функция strftime (в  других БД есть MONTH и YEAR)
sql = '''
SELECT c.customer_nm FROM
(SELECT customer_id, COUNT(*) full_months, strftime("%m", CURRENT_TIMESTAMP) - 1 full_months_now FROM
(SELECT
  customer_id,
  strftime("%m", dt) AS month,
  count(*) sales_qty
from sales
WHERE strftime("%Y", dt) = strftime("%Y", CURRENT_TIMESTAMP)
group by 1,2
having sales_qty >= 3)
GROUP BY customer_id) s
LEFT JOIN customers c ON c.customer_id = s.customer_id
WHERE full_months = full_months_now'''
select(sql)

,customer_nm
0,Дарина Жукова
1,Виктория Савельева


Запрос 2

In [ ]:
sql = '''
SELECT *, ROUND(CAST(store_sales_amt  AS REAL) / company_sales_amt, 2) store_sales_share FROM
(SELECT 
  month_end_dt,
  SUM(store_sales_amt) OVER (PARTITION BY month_end_dt) company_sales_amt,
  store_nm,
  store_sales_amt
 FROM
(SELECT
  DATE(sls.dt, 'start of month', '+1 month', '-1 day') month_end_dt,
  sts.store_nm,
  SUM(amt) store_sales_amt
FROM sales sls
LEFT JOIN stores sts ON sts.store_id = sls.store_id
GROUP BY 1, 2)) LIMIT 20'''
select(sql)

,month_end_dt,company_sales_amt,store_nm,store_sales_amt,store_sales_share
0,2022-02-28,9700,Solity,9700,1.00
1,2022-03-31,5900,Shingo,1900,0.32
2,2022-03-31,5900,Skybit,4000,0.68
3,2022-04-30,57200,Practix,14100,0.25
4,2022-04-30,57200,Skybit,13000,0.23
5,2022-04-30,57200,Stanter,30100,0.53
6,2022-05-31,15000,Practix,11300,0.75
7,2022-05-31,15000,Skybit,3700,0.25
8,2022-06-30,58500,Practix,35000,0.60
9,2022-06-30,58500,Solity,23500,0.40


#Задача 3

Импорт тестовых данных

In [ ]:
id ='1-BPiQwhrbotQgBi1Xta7Iz9j9frmMNnWK3QXpXmY-0Q'
calls= pd.read_csv(f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv')
calls['date'] = pd.to_datetime(calls['date'], dayfirst=True).dt.date
calls.to_sql('calls', con, index=False, if_exists='replace')

9999

Задача 3 запрос 1

In [ ]:
sql = '''
SELECT DISTINCT id_abon FROM calls WHERE roaming'''
select(sql)

,id_abon
0,428
1,379
2,418
3,298
4,369
...,...
492,300
493,439
494,450
495,9


Задача 3 запрос 2

In [ ]:
sql = '''
WITH roam AS 
(SELECT DISTINCT id_abon FROM calls WHERE roaming),
abnt AS
(SELECT DISTINCT id_abon FROM calls)
SELECT * FROM abnt
EXCEPT
SELECT * FROM roam'''
select(sql)

,id_abon
0,1
1,2
2,3


Задача 3 запрос 3

In [ ]:
#запрос содержит эмуляцию таблиц для задания (в блоке CTE) и эмуляцию FULL OUTER JOIN для SQLite (не поддерживается в этой БД)
sql = '''
WITH roam AS 
(SELECT id_abon, SUM(clc) clc FROM calls WHERE roaming GROUP BY id_abon),
abnt AS
(SELECT id_abon, SUM(clc) clc FROM calls GROUP BY id_abon)
SELECT id
FROM
(SELECT a.id_abon id,
         a.clc all_clc,
         r.id_abon,
         r.clc roam_clc
FROM abnt a
LEFT JOIN roam r USING(id_abon)
UNION ALL
SELECT a.id_abon id,
         a.clc all_clc,
         r.id_abon,
         r.clc roam_clc
FROM roam r 
LEFT JOIN abnt a USING(id_abon)
WHERE a.id_abon IS NULL)
WHERE IFNULL(all_clc, 0) - IFNULL(roam_clc, 0) > 0
'''
select(sql)

,id
0,1
1,2
2,3
3,5
4,6
...,...
494,496
495,497
496,498
497,499


Задача 3 запрос 4

In [ ]:
sql = '''
SELECT id_abon
FROM calls
WHERE roaming
GROUP BY id_abon
HAVING COUNT(*)>5
'''
select(sql)

,id_abon
0,4
1,6
2,7
3,8
4,10
...,...
458,496
459,497
460,498
461,499


Задача 3 запрос 5

In [ ]:
sql = '''
SELECT 
  id_abon,
  MIN(date) AS start_date,
  MAX(date) AS end_date
FROM (
  SELECT 
    id_abon,
    date, 
    time, 
    roaming,
    ROW_NUMBER() OVER (PARTITION BY id_abon ORDER BY date, time) - ROW_NUMBER() OVER (PARTITION BY id_abon, roaming ORDER BY date, time) AS session
  FROM calls
) AS t
GROUP BY id_abon, roaming, session
HAVING roaming
'''
select(sql)

,id_abon,start_date,end_date
0,4,2023-01-09,2023-03-22
1,5,2023-01-11,2023-01-11
2,5,2023-02-17,2023-02-17
3,5,2023-03-04,2023-03-04
4,5,2023-03-14,2023-03-14
...,...,...,...
2600,499,2023-03-15,2023-03-16
2601,500,2023-01-17,2023-01-20
2602,500,2023-02-03,2023-02-12
2603,500,2023-02-16,2023-02-28


#Задача 4

SELECT id, surname from t1
WHERE id IN (SELECT id FROM t2 WHERE status = 'выдан')